Временный файлик для работы с ORTools

In [25]:
import collections
from ortools.sat.python import cp_model

In [26]:
jobs = [[[(6, 0), (7, 1), (7, 2)], [(4, 1)], [(8, 0), (6, 1), (7, 2)]],
 [[(18, 1)], [(2, 0), (3, 2)]],
 [[(5, 0), (7, 1)], [(9, 0), (12, 1), (12, 2)]],
 [[(16, 0), (10, 2)]]]

num_jobs = len(jobs)
all_jobs = range(num_jobs)

num_machines = 3
all_machines = range(num_machines)

In [27]:
model = cp_model.CpModel()

In [28]:
horizon = 0
for job in jobs:
    for task in job:
        max_task_duration = 0
        for alternative in task:
            max_task_duration = max(max_task_duration, alternative[0])
        horizon += max_task_duration

In [29]:
horizon

75

In [30]:
intervals_per_resources = collections.defaultdict(list)
starts = {}  # indexed by (job_id, task_id).
presences = {}  # indexed by (job_id, task_id, alt_id).
job_ends: list[cp_model.IntVar] = []

In [31]:
        for job_id in all_jobs:
            job = jobs[job_id]
            num_tasks = len(job)
            previous_end = None
            for task_id in range(num_tasks):
                task = job[task_id]

                min_duration = task[0][0]
                max_duration = task[0][0]

                num_alternatives = len(task)
                all_alternatives = range(num_alternatives)

                for alt_id in range(1, num_alternatives):
                    alt_duration = task[alt_id][0]
                    min_duration = min(min_duration, alt_duration)
                    max_duration = max(max_duration, alt_duration)

                # Create main interval for the task.
                suffix_name = f"_j{job_id}_t{task_id}"
                start = model.new_int_var(0, horizon, "start" + suffix_name)
                duration = model.new_int_var(
                    min_duration, max_duration, "duration" + suffix_name
                )
                end = model.new_int_var(0, horizon, "end" + suffix_name)
                interval = model.new_interval_var(
                    start, duration, end, "interval" + suffix_name
                )

                # Store the start for the solution.
                starts[(job_id, task_id)] = start

                # Add precedence with previous task in the same job.
                if previous_end is not None:
                    model.add(start >= previous_end)
                previous_end = end

                # Create alternative intervals.
                if num_alternatives > 1:
                    l_presences = []
                    for alt_id in all_alternatives:
                        alt_suffix = f"_j{job_id}_t{task_id}_a{alt_id}"
                        l_presence = model.new_bool_var("presence" + alt_suffix)
                        l_start = model.new_int_var(0, horizon, "start" + alt_suffix)
                        l_duration = task[alt_id][0]
                        l_end = model.new_int_var(0, horizon, "end" + alt_suffix)
                        l_interval = model.new_optional_interval_var(
                            l_start, l_duration, l_end, l_presence, "interval" + alt_suffix
                        )
                        l_presences.append(l_presence)

                        # Link the primary/global variables with the local ones.
                        model.add(start == l_start).only_enforce_if(l_presence)
                        model.add(duration == l_duration).only_enforce_if(l_presence)
                        model.add(end == l_end).only_enforce_if(l_presence)

                        # Add the local interval to the right machine.
                        intervals_per_resources[task[alt_id][1]].append(l_interval)

                        # Store the presences for the solution.
                        presences[(job_id, task_id, alt_id)] = l_presence

                    # Select exactly one presence variable.
                    model.add_exactly_one(l_presences)
                else:
                    intervals_per_resources[task[0][1]].append(interval)
                    presences[(job_id, task_id, 0)] = model.new_constant(1)

            if previous_end is not None:
                job_ends.append(previous_end)

In [32]:
for machine_id in all_machines:
    intervals = intervals_per_resources[machine_id]
    if len(intervals) > 1:
        model.add_no_overlap(intervals)

# Makespan objective
makespan = model.new_int_var(0, horizon, "makespan")
model.add_max_equality(makespan, job_ends)
model.minimize(makespan)

# Solve model.
solver = cp_model.CpSolver()

In [33]:
status = solver.solve(model)

In [45]:
ans = [[] for _ in range(num_machines)]

In [46]:
for job_id in all_jobs:
    for task_id, task in enumerate(jobs[job_id]):
        start_value = solver.value(starts[(job_id, task_id)])
        machine: int = -1
        task_duration: int = -1
        selected: int = -1
        for alt_id, alt in enumerate(task):
            if solver.boolean_value(presences[(job_id, task_id, alt_id)]):
                task_duration, machine = alt
                selected = alt_id

                ans[machine].append((start_value, job_id))

In [47]:
ans = list(map(lambda x: [m for t, m in sorted(x)], ans))

In [48]:
ans

[[2, 2, 1], [1, 0, 0], [0, 3]]

In [17]:
if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
    print(f"Optimal objective value: {solver.objective_value}")
    for job_id in all_jobs:
        print(f"Job {job_id}")
        for task_id, task in enumerate(jobs[job_id]):
            start_value = solver.value(starts[(job_id, task_id)])
            machine: int = -1
            task_duration: int = -1
            selected: int = -1
            for alt_id, alt in enumerate(task):
                if solver.boolean_value(presences[(job_id, task_id, alt_id)]):
                    task_duration, machine = alt
                    selected = alt_id
            print(
                f"  task_{job_id}_{task_id} starts at {start_value} (alt"
                f" {selected}, machine {machine}, duration {task_duration})"
            )

print(solver.response_stats())

Optimal objective value: 28.0
Job 0
  task_0_0 starts at 0 (alt 2, machine 2, duration 7)
  task_0_1 starts at 18 (alt 0, machine 1, duration 4)
  task_0_2 starts at 22 (alt 1, machine 1, duration 6)
Job 1
  task_1_0 starts at 0 (alt 0, machine 1, duration 18)
  task_1_1 starts at 18 (alt 1, machine 2, duration 3)
Job 2
  task_2_0 starts at 0 (alt 0, machine 0, duration 5)
  task_2_1 starts at 5 (alt 0, machine 0, duration 9)
Job 3
  task_3_0 starts at 7 (alt 1, machine 2, duration 10)
CpSolverResponse summary:
status: OPTIMAL
objective: 28
best_bound: 28
integers: 51
booleans: 59
conflicts: 2
branches: 198
propagations: 169
integer_propagations: 924
restarts: 54
lp_iterations: 19
walltime: 0.0126121
usertime: 0.0126121
deterministic_time: 0.000593519
gap_integral: 0.0019823
solution_fingerprint: 0x1b05c79d6743487

